In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

%config InlineBackend.figure_format = 'svg'
%matplotlib inline 
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190921.txt', parse_dates=[['DATE', 'TIME']])
df['day_of_week'] = df['DATE_TIME'].dt.weekday_name
df.columns

Index(['DATE_TIME', 'C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION',
       'DESC', 'ENTRIES',
       'EXITS                                                               ',
       'day_of_week'],
      dtype='object')

In [4]:
# Rename exit
df.rename(columns ={"EXITS                                                               ":"EXITS"}, inplace=True)

In [5]:
# Get only regular/ working units. The irregulars could be contributing to outliers
df = df[df['DESC'] == 'REGULAR']

In [6]:
df.describe()

,ENTRIES,EXITS
count,2.042350e+05,2.042350e+05
mean,4.215594e+07,3.490572e+07
std,2.157581e+08,1.996471e+08
min,0.000000e+00,0.000000e+00
25%,2.896500e+05,1.292280e+05
50%,2.038905e+06,1.178385e+06
75%,6.615760e+06,4.534095e+06
max,2.129093e+09,2.123772e+09


In [7]:
# Create dif scores for entries / exits
df['ENTRIES_diff'] = df.ENTRIES.diff()
df['EXITS_diff'] = df.EXITS.diff()
df['TRAFFIC_FLOW'] = df['ENTRIES_diff'] + df['EXITS_diff']

In [8]:
df.columns

Index(['DATE_TIME', 'C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION',
       'DESC', 'ENTRIES', 'EXITS', 'day_of_week', 'ENTRIES_diff', 'EXITS_diff',
       'TRAFFIC_FLOW'],
      dtype='object')

In [9]:
df.head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS,day_of_week,ENTRIES_diff,EXITS_diff,TRAFFIC_FLOW
0,2019-09-14 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198818,2438323,Saturday,NaN,NaN,NaN
1,2019-09-14 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198834,2438325,Saturday,16.0,2.0,18.0
2,2019-09-14 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198847,2438354,Saturday,13.0,29.0,42.0
3,2019-09-14 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198929,2438428,Saturday,82.0,74.0,156.0
4,2019-09-14 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7199125,2438483,Saturday,196.0,55.0,251.0


In [10]:
df['TRAFFIC_FLOW'].describe()

count    2.042340e+05
mean    -4.715751e+01
std      8.230804e+07
min     -3.959976e+09
25%      2.500000e+01
50%      1.780000e+02
75%      4.920000e+02
max      3.959539e+09
Name: TRAFFIC_FLOW, dtype: float64

In [11]:
# set date_time to index
df.set_index('DATE_TIME', inplace = True)

In [12]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS,day_of_week,ENTRIES_diff,EXITS_diff,TRAFFIC_FLOW
DATE_TIME,,,,,,,,,,,,,
2019-09-14 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198818,2438323,Saturday,NaN,NaN,NaN
2019-09-14 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198834,2438325,Saturday,16.0,2.0,18.0
2019-09-14 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198847,2438354,Saturday,13.0,29.0,42.0
2019-09-14 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7198929,2438428,Saturday,82.0,74.0,156.0
2019-09-14 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7199125,2438483,Saturday,196.0,55.0,251.0


In [13]:
df['ENTRIES'].describe()

count    2.042350e+05
mean     4.215594e+07
std      2.157581e+08
min      0.000000e+00
25%      2.896500e+05
50%      2.038905e+06
75%      6.615760e+06
max      2.129093e+09
Name: ENTRIES, dtype: float64

In [14]:
# Replace outliers: delete under zero, replace > 20,000 with mean
# GROUPED BY STATION-- so means are coming by station

df['ENTRIES_diff'] = df.groupby(['STATION']).ENTRIES_diff.transform(
    lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))

In [15]:
df.ENTRIES_diff.describe()

count    204234.000000
mean        188.768041
std         360.117240
min           0.000000
25%          10.000000
50%          84.000000
75%         254.000000
max       19998.000000
Name: ENTRIES_diff, dtype: float64

In [16]:
df['EXITS_diff'] = df.groupby(['STATION']).EXITS_diff.transform(
    lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))

In [17]:
df.EXITS_diff.describe()

count    204234.000000
mean        153.670505
std         372.759533
min           0.000000
25%           9.000000
50%          60.000000
75%         182.458537
max       19987.000000
Name: EXITS_diff, dtype: float64

In [18]:
# Look at station level
gp_station = df.groupby('STATION')

In [75]:
# summing all entries_diff (which is 4 hour segments of entries), summing (so getting across all dates in data),
# showing highest 5 
gp_station['ENTRIES_diff'].sum().sort_values(ascending=False)[:7]

STATION
34 ST-PENN STA     1.050090e+06
GRD CNTRL-42 ST    9.132488e+05
34 ST-HERALD SQ    7.746095e+05
23 ST              7.652739e+05
14 ST-UNION SQ     6.711826e+05
TIMES SQ-42 ST     6.387940e+05
FULTON ST          6.236192e+05
Name: ENTRIES_diff, dtype: float64

### Now that we have the top 5 stations, we can focus on which exits/entries within to focus on. We can also do some kind of analysis by day of week, or time of day

In [89]:
# Making a df with top 5 entries
top_entry_stations = df[df['STATION'].isin(['34 ST-PENN STA','GRD CNTRL-42 ST', '34 ST-HERALD SQ', '23 ST', '14 ST-UNION SQ' ])].nunique()

In [61]:
df.groupby('day_of_week').mean()

,ENTRIES,EXITS,ENTRIES_diff,EXITS_diff,TRAFFIC_FLOW
day_of_week,,,,,
Friday,4.219604e+07,3.486625e+07,213.966204,170.115461,4528.527327
Monday,4.225003e+07,3.500805e+07,201.336146,159.140703,333.984355
Saturday,4.211893e+07,3.486300e+07,161.336389,154.103031,14076.199471
Sunday,4.217439e+07,3.487956e+07,96.746350,82.087208,46.111020
Thursday,4.220531e+07,3.486576e+07,217.504577,171.283393,-20284.364732
Tuesday,4.198997e+07,3.488532e+07,212.798860,167.695397,197.912409
Wednesday,4.215796e+07,3.497263e+07,217.417733,171.140091,818.491196


In [90]:
df.index

DatetimeIndex(['2019-09-14 00:00:00', '2019-09-14 04:00:00',
               '2019-09-14 08:00:00', '2019-09-14 12:00:00',
               '2019-09-14 16:00:00', '2019-09-14 20:00:00',
               '2019-09-15 00:00:00', '2019-09-15 04:00:00',
               '2019-09-15 08:00:00', '2019-09-15 12:00:00',
               ...
               '2019-09-19 09:00:00', '2019-09-19 13:00:00',
               '2019-09-19 17:00:00', '2019-09-19 21:00:00',
               '2019-09-20 01:00:00', '2019-09-20 05:00:00',
               '2019-09-20 09:00:00', '2019-09-20 13:00:00',
               '2019-09-20 17:00:00', '2019-09-20 21:00:00'],
              dtype='datetime64[ns]', name='DATE_TIME', length=204235, freq=None)

In [69]:
monthly_high = df['ENTRIES_diff'].resample("D").max()

In [70]:
monthly_high.max()

19998.0